<a href="https://colab.research.google.com/github/tonystark11/transformer-from-scratch/blob/main/src/translation_transformer_from_scratch_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installations

In [1]:
!pip install -U 'spacy[cuda-autodetect]' -q

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.6/89.6 MB 7.6 MB/s eta 0:00:00


In [4]:
!python -m spacy download en_core_web_sm
!python -m spacy download es_core_news_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ------------------ --------------------- 5.8/12.8 MB 27.0 MB/s eta 0:00:01
     --------------------------------------  12.6/12.8 MB 31.5 MB/s eta 0:00:01
     ---------------------------------------- 12.8/12.8 MB 28.6 MB/s  0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
     ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
     ------------------ --------------------- 6.0/12.9 MB 33.5 MB/s eta 0:00:01
     ------------------------------- ------- 10.5/12.9 MB 32.8 MB/s eta 0:00:01
     ---------------------------------------- 12.9/12.9 MB 21.2 MB/s  0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


# Imports

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy
from functools import partial

from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [7]:
random_seed = 42

# MultiHeadAttention

In [8]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads

        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)

        attn_probs = torch.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_probs, V)
        return output

    def split_heads(self, x):
        batch_size, seq_len, d_model = x.size()
        return x.reshape(batch_size, seq_len, self.num_heads, self.d_k).transpose(1, 2)

    def combine_heads(self, x):
        batch_size, _, seq_len, d_k = x.size()
        return x.transpose(1, 2).reshape(batch_size, seq_len, self.d_model)

    def forward(self, Q, K, V, mask=None):
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))

        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        output = self.W_o(self.combine_heads(attn_output))
        return output


# Position wise Feed Forward Network

In [9]:
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

# Positional Encoding

In [10]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()

        pe = torch.zeros(max_seq_length, d_model, device=device)
        position = torch.arange(0, max_seq_length, dtype=torch.float, device=device).unsqueeze(1)
        div_term = torch.pow(10_000, (-torch.arange(0, d_model, 2, device=device).float() / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        return self.register_buffer('pe', pe.unsqueeze(0))


    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

# Encoder Layer

In [11]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()

        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x


# Decoder Layer

In [12]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()

        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)


    def forward(self, x, enc_output, src_mask, tgt_mask):
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x

# Transformer Model

In [13]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length, device=device), diagonal=1)).bool()
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask

    def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)

        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

        output = self.fc(dec_output)
        return output

# Load Data

In [14]:
import requests

url = "https://www.manythings.org/anki/spa-eng.zip"
filename = "spa-eng.zip"

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)\
 Chrome/58.0.3029.110 Safari/537.3'}

response = requests.get(url, headers=headers, stream=True)
if response.status_code == 200:
    with open(filename, 'wb') as f:
        for chunk in response.iter_content(1024):
            f.write(chunk)
    print("Download completed")
else:
    print(f"Failed to download. Status code: {response.status_code}")


Download completed


In [15]:
import zipfile

# Path to your zip file
zip_path = 'spa-eng.zip'

# Path to extract the contents
extract_to = './unzip_folder'

# Open the zip file in read mode
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("Extraction completed.")


Extraction completed.


In [16]:
with open(r'C:\Users\rahul.g\Documents\transformer-from-scratch\src\unzip_folder\spa.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()


In [17]:
len(lines)

142928

In [18]:
lines[10000]

'He made her cry.\tLe hizo llorar.\tCC-BY 2.0 (France) Attribution: tatoeba.org #1657716 (Spamster) & #2351614 (Shishir)\n'

In [19]:
# Remove everything after the 2nd tab character.
# As we can see above, we only need the first two columns of the data
lines = [line.split('\t') for line in lines]
lines = ['\t'.join(line[:2]) for line in lines]

In [20]:
lines[10000]

'He made her cry.\tLe hizo llorar.'

In [21]:
# Create train, val, test split
train_lines, val_test_lines = train_test_split(lines, test_size=0.2, random_state=random_seed, shuffle=True)
val_lines, test_lines = train_test_split(val_test_lines, test_size=0.5, random_state=random_seed, shuffle=True)

In [22]:
print(len(train_lines))
print(len(val_lines))
print(len(test_lines))

114342
14293
14293


In [23]:
train_lines[0]

"There's a sandwich here.\tHay un sándwich aquí."

In [24]:
val_lines[0]

'Tom cannot drive.\tTom no sabe conducir.'

In [25]:
test_lines[0]

'If you really want to help, please come by 2:30.\tSi realmente quieres ayudar, por favor ven a las 2:30.'

# Preprocess Data

In [26]:
SRC_LANGUAGE = "en"
TGT_LANGUAGE = "es"

In [27]:
tokenizer = {}
tokenizer[SRC_LANGUAGE] = get_tokenizer("spacy", "en_core_web_sm")
tokenizer[TGT_LANGUAGE] = get_tokenizer("spacy", "es_core_news_sm")

## Create Dataset

In [28]:
class SentencePairDataset(Dataset):
    def __init__(self, lines, src_tokenizer, tgt_tokenizer):
        super(SentencePairDataset, self).__init__()

        self.lines = lines
        self.src_tokenizer = src_tokenizer
        self.tgt_tokenizer = tgt_tokenizer

    def __len__(self):
        return len(self.lines)

    def __getitem__(self, idx):
        line = self.lines[idx]

        src, tgt = line.split('\t')
        src_tokens = self.src_tokenizer(src)
        tgt_tokens = self.tgt_tokenizer(tgt)

        return src_tokens, tgt_tokens

In [29]:
train_ds = SentencePairDataset(train_lines, tokenizer[SRC_LANGUAGE], tokenizer[TGT_LANGUAGE])
val_ds = SentencePairDataset(val_lines, tokenizer[SRC_LANGUAGE], tokenizer[TGT_LANGUAGE])
test_ds = SentencePairDataset(test_lines, tokenizer[SRC_LANGUAGE], tokenizer[TGT_LANGUAGE])

In [30]:
# Length of longest src sequence
print(max(len(x[0]) for x in train_ds))
print(max(len(x[0]) for x in val_ds))
print(max(len(x[0]) for x in test_ds))

81
40
31


In [31]:
# Length of longest tgt sequence
print(max(len(x[1]) for x in train_ds))
print(max(len(x[1]) for x in val_ds))
print(max(len(x[1]) for x in test_ds))

78
44
30


In [32]:
next(iter(train_ds))

(['There', "'s", 'a', 'sandwich', 'here', '.'],
 ['Hay', 'un', 'sándwich', 'aquí', '.'])

## Create Vocabulary

In [33]:
vocab = {}

In [34]:
src_vocab_size = 10_000
tgt_vocab_size = 10_000
max_seq_len = 100

PAD_IDX = 0
UNK_IDX = 1
BOS_IDX = 2
EOS_IDX = 3

special_symbols = ['<PAD>', '<UNK>', '<BOS>', '<EOS>']

In [35]:
def yield_tokens(dataset, lang_idx=0):
    n = len(dataset)
    i = 0

    while i < n:
        yield dataset[i][lang_idx]
        i += 1

In [36]:
src_iterator = yield_tokens(train_ds, lang_idx=0)
tgt_iterator = yield_tokens(train_ds, lang_idx=1)

In [37]:
vocab[SRC_LANGUAGE] = build_vocab_from_iterator(
    src_iterator,
    min_freq=1,
    specials=special_symbols,
    special_first=True,
    max_tokens=src_vocab_size,
)

In [38]:
vocab[TGT_LANGUAGE] = build_vocab_from_iterator(
    tgt_iterator,
    min_freq=1,
    specials=special_symbols,
    special_first=True,
    max_tokens=tgt_vocab_size,
)

In [39]:
vocab[SRC_LANGUAGE].set_default_index(UNK_IDX)
vocab[TGT_LANGUAGE].set_default_index(UNK_IDX)

In [40]:
vocab[SRC_LANGUAGE]['hello']

2148

In [41]:
vocab[TGT_LANGUAGE]['Hola']

2626

In [42]:
def collate_fn(batch, vocab):
    batch_size = len(batch)
    srcs, tgts = zip(*batch)
    src_vectors = torch.zeros((batch_size, max_seq_len), dtype=torch.long, device=device)
    tgt_vectors = torch.zeros((batch_size, max_seq_len), dtype=torch.long, device=device)

    for i in range(batch_size):
        src_vectors[i] = torch.tensor(([BOS_IDX] + vocab[SRC_LANGUAGE](srcs[i]) + [EOS_IDX] + [0] * (max_seq_len - len(srcs[i])))[:max_seq_len], dtype=torch.long, device=device)
        tgt_vectors[i] = torch.tensor(([BOS_IDX] + vocab[TGT_LANGUAGE](tgts[i]) + [EOS_IDX] + [0] * (max_seq_len - len(tgts[i])))[:max_seq_len], dtype=torch.long, device=device)

    return src_vectors, tgt_vectors

In [43]:
train_dataloader = DataLoader(train_ds, batch_size=64, shuffle=True, collate_fn=partial(collate_fn, vocab=vocab))
val_dataloader = DataLoader(val_ds, batch_size=64, shuffle=True, collate_fn=partial(collate_fn, vocab=vocab))
test_dataloader = DataLoader(test_ds, batch_size=64, shuffle=True, collate_fn=partial(collate_fn, vocab=vocab))

In [44]:
src_vocab_size = 10_000
tgt_vocab_size = 10_000
d_model = 256
num_heads = 2
num_layers = 3
d_ff = 512
max_seq_length = 100
dropout = 0.1
num_epochs = 3

transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout).to(device)

criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

for epoch in range(num_epochs):
    print(f"Epoch: {epoch+1}\n------------------------------")
    transformer.train()
    for data in train_dataloader:
        src_data, tgt_data = data
        optimizer.zero_grad()
        output = transformer(src_data, tgt_data[:, :-1])
        loss = criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_data[:, 1:].contiguous().view(-1))
        loss.backward()
        optimizer.step()
        print(f"Epoch: {epoch+1}, Training Loss: {loss.item()}")

    transformer.eval()
    with torch.no_grad():
        for data in val_dataloader:
            src_data, tgt_data = data
            output = transformer(src_data, tgt_data[:, :-1])
            loss = criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_data[:, 1:].contiguous().view(-1))
            print(f"Epoch: {epoch+1}, Validation Loss: {loss.item()}")

    torch.save(transformer.state_dict(), f'./transformer_state_dict_epoch_{epoch+1}')

Epoch: 1
------------------------------
Epoch: 1, Training Loss: 9.309993743896484
Epoch: 1, Training Loss: 9.143234252929688
Epoch: 1, Training Loss: 8.865801811218262
Epoch: 1, Training Loss: 8.617685317993164
Epoch: 1, Training Loss: 8.465410232543945
Epoch: 1, Training Loss: 8.318875312805176
Epoch: 1, Training Loss: 8.179346084594727
Epoch: 1, Training Loss: 8.004155158996582
Epoch: 1, Training Loss: 7.983952045440674
Epoch: 1, Training Loss: 7.95011043548584
Epoch: 1, Training Loss: 7.701634883880615
Epoch: 1, Training Loss: 7.790953636169434
Epoch: 1, Training Loss: 7.780338287353516
Epoch: 1, Training Loss: 7.67399787902832
Epoch: 1, Training Loss: 7.690968036651611
Epoch: 1, Training Loss: 7.561918258666992
Epoch: 1, Training Loss: 7.549985885620117
Epoch: 1, Training Loss: 7.631964683532715
Epoch: 1, Training Loss: 7.414304733276367
Epoch: 1, Training Loss: 7.434693813323975
Epoch: 1, Training Loss: 7.432102203369141
Epoch: 1, Training Loss: 7.441831588745117
Epoch: 1, Traini

In [45]:
transformer.eval()
with torch.no_grad():
    for data in test_dataloader:
        src_data, tgt_data = data
        output = transformer(src_data, tgt_data[:, :-1])
        loss = criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_data[:, 1:].contiguous().view(-1))
        print(f"Test Loss: {loss.item()}")

Test Loss: 1.9633657932281494
Test Loss: 2.136871337890625
Test Loss: 2.209472417831421
Test Loss: 2.5739591121673584
Test Loss: 1.8896713256835938
Test Loss: 2.049983501434326
Test Loss: 1.9989005327224731
Test Loss: 1.9682408571243286
Test Loss: 2.2493555545806885
Test Loss: 1.7985692024230957
Test Loss: 1.91669762134552
Test Loss: 2.1654419898986816
Test Loss: 1.9810129404067993
Test Loss: 2.275704860687256
Test Loss: 1.9295533895492554
Test Loss: 2.298708915710449
Test Loss: 2.0952014923095703
Test Loss: 2.018160820007324
Test Loss: 2.206990957260132
Test Loss: 2.076070547103882
Test Loss: 2.231081485748291
Test Loss: 2.1140525341033936
Test Loss: 2.2065725326538086
Test Loss: 2.185159921646118
Test Loss: 1.9285701513290405
Test Loss: 2.2580907344818115
Test Loss: 2.3661727905273438
Test Loss: 2.2734856605529785
Test Loss: 2.06719708442688
Test Loss: 2.192049741744995
Test Loss: 2.1346540451049805
Test Loss: 2.2178053855895996
Test Loss: 1.7609453201293945
Test Loss: 1.967291116714

# Inference

In [56]:
model_path = "./transformer_state_dict_epoch_3"
state_dict = torch.load(model_path)

src_vocab_size = 10_000
tgt_vocab_size = 10_000
d_model = 256
num_heads = 2
num_layers = 3
d_ff = 512
max_seq_length = 100
dropout = 0.1
num_epochs = 3

transformer_loaded = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout).to(device)
transformer_loaded.load_state_dict(state_dict)

<All keys matched successfully>

In [57]:
def translate(src):
    src_tokens = tokenizer[SRC_LANGUAGE](src)
    tgt_tokens = ["<BOS>"]

    src_vectors = torch.tensor(([BOS_IDX] + vocab[SRC_LANGUAGE](src_tokens) + [EOS_IDX] + [0] * (max_seq_len - len(src_tokens)))[:max_seq_len], dtype=torch.long, device=device).unsqueeze(0)

    for i in range(max_seq_len):
        tgt_vectors = torch.tensor((vocab[TGT_LANGUAGE](tgt_tokens) + [0] * (max_seq_len - len(tgt_tokens)))[:max_seq_len], dtype=torch.long, device=device).unsqueeze(0)
        output = transformer(src_vectors, tgt_vectors)
        idx = torch.argmax(nn.functional.softmax(output, dim=2)[0][i]).item()
        tgt_tokens.append(vocab[TGT_LANGUAGE].lookup_token(idx))

        if idx == EOS_IDX:
            break

    return " ".join(tgt_tokens).replace("<BOS>", "").replace("<EOS>", "").replace("<PAD>", "").strip()

In [58]:
translate("Hello, I am a teacher.")

'Por favor , estoy profesor .'

In [59]:
translate("My name is John.")

'Mi nombre es John .'

In [60]:
translate("I am learning Spanish.")

'Estoy aprendiendo .'

In [61]:
translate("I eat apples.")

'Me voy a manzanas .'

In [62]:
translate("I have three books and two pens.")

'Tengo tres libros y libros .'

In [63]:
translate("Do you work in an office?")

'¿ Quieres trabajar en un oficina ?'

In [64]:
translate("How are you?")

'¿ Cómo estás ?'

In [65]:
eng, spa = test_lines[0].split('\t')
print(eng)
print(spa)
translate(eng)

If you really want to help, please come by 2:30.
Si realmente quieres ayudar, por favor ven a las 2:30.


'Si quieres quieres ayudar , por favor a las 2:30 .'

In [66]:
eng, spa = test_lines[500].split('\t')
print(eng)
print(spa)
translate(eng)

I was stunned by what happened.
Yo estaba anonadado por lo que había ocurrido.


'Estaba de hacer lo que pasó .'

In [67]:
eng, spa = train_lines[1000].split('\t')
print(eng)
print(spa)
translate(eng)

This one has a lot of advantages over that one.
Esta tiene un montón de ventajas por sobre aquella.


'Este es mucho de mucha <UNK> .'

In [68]:
eng, spa = train_lines[10000].split('\t')
print(eng)
print(spa)
translate(eng)

Tell me what Tom said.
Cuéntame lo que dijo Tom.


'Dime lo que dijo Tom .'

# Export model and vocabulary

In [69]:
torch.save(vocab[SRC_LANGUAGE], "./vocab-english")
torch.save(vocab[TGT_LANGUAGE], "./vocab-spanish")

In [70]:
torch.save(tokenizer[SRC_LANGUAGE], "./tokenizer-english")
torch.save(tokenizer[TGT_LANGUAGE], "./tokenizer-spanish")

In [71]:
torch.save(transformer, "./transformer_model")